In [112]:
import requests
from bs4 import BeautifulSoup
import os


#url = 'http://www.cartoonmad.com/comic/5540.html'

header_s={
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    'Cookie': 'ASPSESSIONIDAQQSDBBQ=ILCIIGDCJJDGCNDPPJHNDCHI; __utma=42881207.196417283.1522300799.1522300799.1522300799.1; __utmc=42881207; __utmz=42881207.1522300799.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utmt=1; chname=%7C%A5j%A8%A3%A6P%BE%C7%A6%B3%A5%E6%ACy%BB%D9%C3%AA%AFg%7C%A4C%AD%EC%B8o%7C%AE%FC%B8%E9%A4%FD; chid=%7C05540%7C02504%7C01152; __utmb=42881207.6.10.1522300799',
    'Host': 'www.cartoonmad.com',
    'Pragma': 'no-cache',
    'Referer': 'http://www.cartoonmad.com/comic/554000972009001.html',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'
}

def get_imgs_links(url, header_s):
    imgs_links = []
    res = requests.get(url, headers=header_s)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, "lxml")
    selector_list = soup.select('select option[value]')
    imgs_num = len(selector_list)
    print('this monga total page number = %s'%str(imgs_num))
    url_template = soup.select('a img[src^=http://web.cartoonmad.com]')[0]['src']
    pat = re.findall('\d{3}.jpg', url_template)[0]
    url_template = url_template.replace(pat, '')
    print('url template : %s'%url_template)
    for num in range(1, imgs_num+1):
        num_str = '{:03}'.format(num)+'.jpg'
        link = url_template+num_str
        imgs_links.append(link)
        print(link)
    return imgs_links


def get_one_monga(url, header_s):
    imgs_links = get_imgs_links(url, header_s)
    res = requests.get(url, headers=header_s)
    res.encoding = 'big5'
    soup = BeautifulSoup(res.text, "lxml")
    title = soup.select('center li a')[0].text
    epi = soup.select('center li a')[1].text.replace(' 話', '')
    dir_name = title+epi
    print('this monga title is : '+dir_name)
    os.makedirs('crawl_mongas/'+dir_name, exist_ok=True)
    for num in range(len(imgs_links)):
        filename = 'crawl_mongas/'+dir_name+'/'+'p_'+str(num+1)+'.jpg'
        img_data = requests.get(imgs_links[num], headers=header_s).content
        with open(filename, 'wb') as f:
            f.write(img_data)
        print('[INFO]Done download page %s'%filename)
    print('[INFO]Done download monga %s'%dir_name)

    
def get_series_mongas(url, header_s):
    res = requests.get(url, headers=header_s)
    res.encoding = 'big5'
    soup = BeautifulSoup(res.text, "lxml")
    selector_list = soup.select('#info table tr td a[href^="/comic/"]')
    epi_num = len(selector_list)
    print('[START]this monga has %s epis'%str(epi_num))
    for num in range(epi_num):
        img_pg_link = 'http://www.cartoonmad.com'+selector_list[num]['href']
        print('image page link = %s'%img_pg_link)
        get_one_monga(img_pg_link, header_s)
    print('Done download all mongas!')



In [113]:
url = 'http://www.cartoonmad.com/comic/5264.html'
get_series_mongas(url, header_s)

[START]this monga has 21 epis
image page link = http://www.cartoonmad.com/comic/526400012051001.html
this monga total page number = 51
url template : http://web.cartoonmad.com/c8kcs73hv62/5264/001/
http://web.cartoonmad.com/c8kcs73hv62/5264/001/001.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/001/002.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/001/003.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/001/004.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/001/005.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/001/006.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/001/007.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/001/008.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/001/009.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/001/010.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/001/011.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/001/012.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/001/013.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/001/014.jpg
http://web.cartoonmad.com/c8kcs7

this monga title is : 哥布林殺手漫畫2
[INFO]Done download page crawl_mongas/哥布林殺手漫畫2/p_1.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫2/p_2.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫2/p_3.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫2/p_4.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫2/p_5.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫2/p_6.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫2/p_7.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫2/p_8.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫2/p_9.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫2/p_10.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫2/p_11.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫2/p_12.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫2/p_13.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫2/p_14.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫2/p_15.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫2/p_16.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫2/p_17.jpg
[INFO]Done download page c

this monga title is : 哥布林殺手漫畫4
[INFO]Done download page crawl_mongas/哥布林殺手漫畫4/p_1.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫4/p_2.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫4/p_3.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫4/p_4.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫4/p_5.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫4/p_6.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫4/p_7.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫4/p_8.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫4/p_9.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫4/p_10.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫4/p_11.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫4/p_12.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫4/p_13.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫4/p_14.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫4/p_15.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫4/p_16.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫4/p_17.jpg
[INFO]Done download page c

this monga title is : 哥布林殺手漫畫6
[INFO]Done download page crawl_mongas/哥布林殺手漫畫6/p_1.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫6/p_2.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫6/p_3.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫6/p_4.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫6/p_5.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫6/p_6.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫6/p_7.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫6/p_8.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫6/p_9.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫6/p_10.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫6/p_11.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫6/p_12.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫6/p_13.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫6/p_14.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫6/p_15.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫6/p_16.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫6/p_17.jpg
[INFO]Done download page c

this monga title is : 哥布林殺手漫畫8
[INFO]Done download page crawl_mongas/哥布林殺手漫畫8/p_1.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫8/p_2.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫8/p_3.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫8/p_4.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫8/p_5.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫8/p_6.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫8/p_7.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫8/p_8.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫8/p_9.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫8/p_10.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫8/p_11.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫8/p_12.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫8/p_13.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫8/p_14.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫8/p_15.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫8/p_16.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫8/p_17.jpg
[INFO]Done download page c

[INFO]Done download page crawl_mongas/哥布林殺手漫畫10/p_2.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫10/p_3.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫10/p_4.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫10/p_5.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫10/p_6.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫10/p_7.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫10/p_8.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫10/p_9.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫10/p_10.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫10/p_11.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫10/p_12.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫10/p_13.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫10/p_14.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫10/p_15.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫10/p_16.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫10/p_17.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫10/p_18.jpg
[INFO]Done download page crawl_mongas/哥

[INFO]Done download page crawl_mongas/哥布林殺手漫畫12/p_21.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫12/p_22.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫12/p_23.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫12/p_24.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫12/p_25.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫12/p_26.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫12/p_27.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫12/p_28.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫12/p_29.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫12/p_30.jpg
[INFO]Done download monga 哥布林殺手漫畫12
image page link = http://www.cartoonmad.com/comic/526400132033001.html
this monga total page number = 33
url template : http://web.cartoonmad.com/c8kcs73hv62/5264/013/
http://web.cartoonmad.com/c8kcs73hv62/5264/013/001.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/013/002.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/013/003.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/013/004.jpg
http:/

this monga title is : 哥布林殺手漫畫15
[INFO]Done download page crawl_mongas/哥布林殺手漫畫15/p_1.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫15/p_2.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫15/p_3.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫15/p_4.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫15/p_5.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫15/p_6.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫15/p_7.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫15/p_8.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫15/p_9.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫15/p_10.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫15/p_11.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫15/p_12.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫15/p_13.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫15/p_14.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫15/p_15.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫15/p_16.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫15/p_17.jpg
[INFO]Do

[INFO]Done download page crawl_mongas/哥布林殺手漫畫17/p_6.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫17/p_7.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫17/p_8.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫17/p_9.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫17/p_10.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫17/p_11.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫17/p_12.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫17/p_13.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫17/p_14.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫17/p_15.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫17/p_16.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫17/p_17.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫17/p_18.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫17/p_19.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫17/p_20.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫17/p_21.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫17/p_22.jpg
[INFO]Done download page crawl_mong

[INFO]Done download page crawl_mongas/哥布林殺手漫畫19/p_19.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫19/p_20.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫19/p_21.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫19/p_22.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫19/p_23.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫19/p_24.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫19/p_25.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫19/p_26.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫19/p_27.jpg
[INFO]Done download page crawl_mongas/哥布林殺手漫畫19/p_28.jpg
[INFO]Done download monga 哥布林殺手漫畫19
image page link = http://www.cartoonmad.com/comic/526400202030001.html
this monga total page number = 30
url template : http://web.cartoonmad.com/c8kcs73hv62/5264/020/
http://web.cartoonmad.com/c8kcs73hv62/5264/020/001.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/020/002.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/020/003.jpg
http://web.cartoonmad.com/c8kcs73hv62/5264/020/004.jpg
http:/